In [43]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

In [44]:
civic = pd.read_csv("Civic-142A-Fall24.csv")

In [45]:
train = civic[civic["Year"] < 2020]
test = civic[civic["Year"] >= 2020]
train.head()

MonthNumeric MonthFactor  Year  CivicSales  Unemployment  CivicQueries  \
0             1     January  2014       21824           6.6            66   
1             2    February  2014       21575           6.7            69   
2             3       March  2014       27697           6.7            72   
3             4       April  2014       27611           6.2            69   
4             5         May  2014       36089           6.3            69   

    CPIAll  CPIEnergy  MilesTraveled  
0  235.288    250.340         246531  
1  235.547    249.925         249499  
2  236.028    249.961         251120  
3  236.468    249.864         251959  
4  236.918    249.213         252289

In [46]:
#list of vars to subset
xVars = ["CivicQueries","CPIAll"]

# filter taining and test into independent vars list
Xtrain = train[xVars]
Xtest = test[xVars]

#create target vectors to predict Sales
Ytrain = train[["CivicSales"]]
Ytest = test[["CivicSales"]]

#add Constants
Xtrain = sm.add_constant(Xtrain)
Xtest = sm.add_constant(Xtest)


In [47]:
model = sm.OLS(Ytrain, Xtrain)

In [48]:
results = model.fit()

In [49]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     22.51
Date:                Mon, 27 Jan 2025   Prob (F-statistic):           2.98e-08
Time:                        21:26:57   Log-Likelihood:                -689.88
No. Observations:                  72   AIC:                             1386.
Df Residuals:                      69   BIC:                             1393.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         5.713e+04   1.45e+04      3.953   

In [50]:
test.head(5)

MonthNumeric MonthFactor  Year  CivicSales  Unemployment  CivicQueries  \
72             1     January  2020       20054           3.5            74   
73             2    February  2020       25617           3.5            76   
74             3       March  2020       18273           4.4            66   
75             4       April  2020       13410          14.8            57   
76             5         May  2020       27244          13.3            78   

     CPIAll  CPIEnergy  MilesTraveled  
72  259.037    219.917         285452  
73  259.248    214.991         284040  
74  258.124    201.747         224870  
75  256.092    183.034         167174  
76  255.868    179.281         204304

In [51]:
modelMonths = smf.ols("CivicSales ~ CivicQueries + CPIAll + CPIEnergy + Unemployment + MilesTraveled + C(MonthFactor)", data = train)

In [52]:
results2 = modelMonths.fit()

In [53]:
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     11.19
Date:                Mon, 27 Jan 2025   Prob (F-statistic):           5.05e-12
Time:                        21:26:57   Log-Likelihood:                -655.83
No. Observations:                  72   AIC:                             1346.
Df Residuals:                      55   BIC:                             1384.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [54]:
model3 = smf.ols("CivicSales ~ CivicQueries + CPIAll + C(MonthFactor)", data = train).fit()

In [55]:
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     13.79
Date:                Mon, 27 Jan 2025   Prob (F-statistic):           2.80e-13
Time:                        21:26:57   Log-Likelihood:                -657.24
No. Observations:                  72   AIC:                             1342.
Df Residuals:                      58   BIC:                             1374.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [56]:
test.reset_index(drop=True, inplace=True)
preds = model3.predict(test)

In [57]:
SSR = np.sum((test["CivicSales"] - preds) ** 2)
y_mean = np.mean(test["CivicSales"])

In [58]:
TSS = np.sum((test["CivicSales"] - y_mean) ** 2)

In [59]:
OSR2 = 1 - (SSR/TSS)

In [60]:
OSR2

-0.09021353946711175

In [61]:
CCI = pd.read_csv("UMCSENT (1).csv")

In [62]:
CCI
civic2 = pd.concat([civic, CCI["UMCSENT"]],axis = 1)
trainCCI = civic2[civic2["Year"] < 2020]
testCCI = civic2[civic2["Year"] >= 2020]

In [63]:
model4 = smf.ols("CivicSales ~ UMCSENT + C(MonthFactor) + CivicQueries + CPIAll", data = trainCCI).fit()

In [64]:
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:             CivicSales   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     12.66
Date:                Mon, 27 Jan 2025   Prob (F-statistic):           9.61e-13
Time:                        21:26:57   Log-Likelihood:                -657.09
No. Observations:                  72   AIC:                             1344.
Df Residuals:                      57   BIC:                             1378.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [65]:
preds4 = model4.predict(testCCI)

In [66]:
SSR_model4 = np.sum((testCCI["CivicSales"] - preds4) ** 2)
y_mean_model4 = np.mean(testCCI["CivicSales"])
TSS_model4 = np.sum((testCCI["CivicSales"]-y_mean_model4)**2)

In [67]:
RSquared_model4 = 1 - (SSR_model4/TSS_model4)

In [68]:
RSquared_model4

-0.4579339533332556